In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import RandomHorizontalFlip

In [ ]:
cv2.calibrateCamera()

## Multi Scale Crop

In [103]:
def crop_corner_center(source_img, image_list=None, if_main=False):
    return_images = []
    img_height, img_width = source_img.shape[0], source_img.shape[1]
    
    if if_main:
        crop_factor = 0.9
        corner_width, corner_height = img_width - int(img_width * crop_factor), img_height - int(img_height * crop_factor)
        center_width, center_height = int((img_width - int(img_width * crop_factor)) / 2), int((img_height - int(img_height * crop_factor)) / 2)
        
        crop_positions = [
            (0, img_width - corner_width, 0, img_height - corner_height),
            (corner_width, img_width, 0, img_height - corner_height),
            (0, img_width - corner_width, corner_height, img_height),
            (corner_width, img_width, corner_height, img_height),
            (center_width, img_width - center_width, center_height, img_height - center_height)
        ]
        
        # augmentation loop
        for c in crop_positions:
            # aug image list
            images = []
            
            # crop - 3 x 5
            cropped_img = source_img[c[0]:c[1], c[2]:c[3], :]
    
            # crop - 3 x 5 x 5
            crop_corner_center(cropped_img, image_list=images, if_main=False)
            
            # Flip - 3 x 5 x 5 x 2
            for aug_img in images:                
                fliped_img = cv2.flip(np.array(aug_img), 1)
                
                # save
                return_images.append(aug_img)
                return_images.append(fliped_img)
    else:
        corner_width, corner_height = img_width - 224, img_height - 224
        center_width, center_height = int((img_width - 224) / 2), int((img_height - 224) / 2)
        
        crop_positions = [
            (0, img_width - corner_width, 0, img_height - corner_height),
            (corner_width, img_width, 0, img_height - corner_height),
            (0, img_width - corner_width, corner_height, img_height),
            (corner_width, img_width, corner_height, img_height),
            (center_width, img_width - center_width, center_height, img_height - center_height)
        ]
        
        for c in crop_positions:
            # crop
            cropped_img = source_img[c[0]:c[1], c[2]:c[3], :]
            image_list.append(cropped_img)
        
        return image_list
    
    return return_images

In [104]:
# open image - cv2
img = cv2.imread('datasets/flowers-102/jpg/image_00001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# multi scale
images = []
for scale in [256, 384, 512]:
    img = cv2.resize(img, (scale, scale))

    # crop corner and center position
    aug_images = crop_corner_center(img, image_list=None, if_main=True)
    
    images.extend(aug_images)